In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from collections import Counter
import re
import unicodedata as ud
from nltk.corpus import wordnet as wn
from nltk.corpus import words as w
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import os
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

In [2]:
df = pd.read_pickle('/Users/isdeniz/Documents/msc-notebooks/msc/label_mapping/msc_dataset_mapped_labels.pkl')
print(df.shape)

(190436, 42)


In [3]:
df.head()

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,user,labels,state,locked,assignee,assignees,milestone,comments,created_at,updated_at,closed_at,author_association,active_lock_reason,body,reactions,timeline_url,performed_via_github_app,state_reason,score,repository_name,issue/pr,created_at_timestamp,closed_at_timestamp,issue_resolution_time,open_since,labels_id,labels_node_id,labels_name,labels_color,labels_default,labels_description,mapLabel
0,https://api.github.com/repos/facebook/react-native/issues/2,https://api.github.com/repos/facebook/react-native,['https://api.github.com/repos/facebook/react-native/labels/Resolution:%20Locked'],https://api.github.com/repos/facebook/react-native/issues/2/comments,https://api.github.com/repos/facebook/react-native/issues/2/events,https://github.com/facebook/react-native/issues/2,55988530,MDU6SXNzdWU1NTk4ODUzMA==,2,`pthread.h` not found,"{'login': 'jlongster', 'id': 17031, 'node_id': 'MDQ6VXNlcjE3MDMx', 'avatar_url': 'https://avatars.githubusercontent.com/u/17031?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/jlongster', 'html_url': 'https://github.com/jlongster', 'followers_url': 'https://api.github.com/users/jlongster/followers', 'following_url': 'https://api.github.com/users/jlongster/following{/other_user}', 'gists_url': 'https://api.github.com/users/jlongster/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/jlongster/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/jlongster/subscriptions', 'organizations_url': 'https://api.github.com/users/jlongster/orgs', 'repos_url': 'https://api.github.com/users/jlongster/repos', 'events_url': 'https://api.github.com/users/jlongster/events{/privacy}', 'received_events_url': 'https://api.github.com/users/jlongster/received_events', 'type': 'User', 'site_admin': False}","[{'id': 998095527, 'node_id': 'MDU6TGFiZWw5OTgwOTU1Mjc=', 'url': 'https://api.github.com/repos/facebook/react-native/labels/Resolution:%20Locked', 'name': 'Resolution: Locked', 'color': 'ffffff', 'default': False, 'description': 'This issue was locked by the bot.'}]",closed,True,None,[],None,2,2015-01-30T02:54:11Z,2018-07-23T05:02:58Z,2015-01-31T00:04:06Z,NONE,resolved,"To build on my machine (OS X 10.9.3) targeting iOS SDK 7.1 (only one available on my machine), I had to change line 7 in `/ReactKit/Modules/RCTUIManager.m` from `#import <pthread/pthread.h>` to `#import <pthread.h>` to get it to compile.\n","{'url': 'https://api.github.com/repos/facebook/react-native/issues/2/reactions', 'total_count': 0, '+1': 0, '-1': 0, 'laugh': 0, 'hooray': 0, 'confused': 0, 'heart': 0, 'rocket': 0, 'eyes': 0}",https://api.github.com/repos/facebook/react-native/issues/2/timeline,None,completed,1.0,facebook/react-native,issue,2015-01-30 02:54:11,2015-01-31 00:04:06,0.881887,NaN,[998095527],['MDU6TGFiZWw5OTgwOTU1Mjc='],['Resolution: Locked'],['ffffff'],[False],['This issue was locked by the bot.'],other
1,https://api.github.com/repos/facebook/react-native/issues/4,https://api.github.com/repos/facebook/react-native,['https://api.github.com/repos/facebook/react-native/labels/Resolution:%20Locked'],https://api.github.com/repos/facebook/react-native/issues/4/comments,https://api.github.com/repos/facebook/react-native/issues/4/events,https://github.com/facebook/react-native/issues/4,56060794,MDU6SXNzdWU1NjA2MDc5NA==,4,Set UIStatusBarStyleLightContent,"{'login': 'nick', 'id': 939, 'node_id': 'MDQ6VXNlcjkzOQ==', 'avatar_url': 'https://avatars.githubusercontent.com/u/939?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/nick', 'html_url': 'https://github.com/nick', 'followers_url': 'https://api.github.com/users/nick/followers', 'following_url': 'https://api.github.com/users/nick/following{/other_user}', 'gists_url': 'https://api.github.com/users/nick/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/nick/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/nick/subsc

Included Labels: bug - enhancement - question 

In [4]:
enhancement_labels = {'enhancement', 'feature', 'proposal', 'improvement', 'suggestion'}
df['mapLabel'] = df['mapLabel'].replace(enhancement_labels, 'enhancement')

question_labels = {'question', 'discussion'}
df['mapLabel'] = df['mapLabel'].replace(question_labels, 'question')

included_labels = {'bug', 'enhancement', 'question'}
df_filtered = df[df['mapLabel'].isin(included_labels)]
df_filtered.reset_index(drop=True, inplace=True)

df_filtered['mapLabel'].value_counts()

mapLabel
bug            55945
enhancement    36369
question       12337
Name: count, dtype: int64

In [5]:
value_counts_with_percentage = df_filtered['mapLabel'].value_counts(normalize = True) * 100
value_counts_with_percentage

mapLabel
bug            53.458639
enhancement    34.752654
question       11.788707
Name: proportion, dtype: float64

Train Test Split

In [6]:
random_seed = 42
train, test = train_test_split(df_filtered, test_size = 0.1, train_size = 0.9, stratify = df_filtered['mapLabel'], random_state = random_seed)
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)
print("Train set:", train.shape)
print("Test set:", test.shape)

Train set: (94185, 42)
Test set: (10466, 42)


In [7]:
train['mapLabel'].value_counts()

mapLabel
bug            50350
enhancement    32732
question       11103
Name: count, dtype: int64

In [8]:
test['mapLabel'].value_counts()

mapLabel
bug            5595
enhancement    3637
question       1234
Name: count, dtype: int64

In [9]:
train['original_issue_title'] = train['title']
train['original_issue_body'] = train['body']

train.rename(columns={'title': 'preprocessed_title'}, inplace=True)
train.rename(columns={'body': 'preprocessed_body'}, inplace=True)

test['original_issue_title'] = test['title']
test['original_issue_body'] = test['body']

test.rename(columns={'title': 'preprocessed_title'}, inplace=True)
test.rename(columns={'body': 'preprocessed_body'}, inplace=True)

In [10]:
# label= 'issue_label'
label = 'mapLabel'

# time = 'issue_created_at'
time = 'created_at'

# repo = 'repository_url'
repo = 'repository_name'

title = 'preprocessed_title'
body = 'preprocessed_body'

# author = 'issue_author_association'
author = 'author_association'

# url = 'issue_url'
url = 'url'

#label_col = 'labels'
label_col = 'labels_cat'
text_col = 'text'

max_title = 30
max_body = 170
# punctuations = '!"$%&\()*,/:;<=>[\\]^`{|}~+#@-`' below line is added since this line (original) gives SyntaxWarning: invalid escape sequence '\('
punctuations = r'!"$%&\()*,/:;<=>[\\]^`{|}~+#@-`'
ascii_regex = re.compile(r'[^\x00-\x7f]')
issue_regex = re.compile(r'#[0-9]+')
function_regex = re.compile(r'[a-zA-Z][a-zA-Z0-9_.]*\([a-zA-Z0-9_, ]*\)')

first deduplicate the TRAINING dataset based on issue URls

In [11]:
dedup_train = train.sort_values(url).drop_duplicates(subset=[url]).copy()
print('Number of dropped issue duplications: ' , train.shape[0] - dedup_train.shape[0])

dedup_train[title] = dedup_train[title].astype(str)
dedup_train[body] = dedup_train[body].astype(str)
dedup_train[author] = dedup_train[author].astype(str)
dedup_train[time] = dedup_train[time].astype(str)
dedup_train[repo] = dedup_train[repo].astype(str)

test[title] = test[title].astype(str)
test[body] = test[body].astype(str)
test[author] = test[author].astype(str)
test[time] = test[time].astype(str)
test[repo] = test[repo].astype(str)

Number of dropped issue duplications:  0


text normalization on text columns of issues

In [12]:
print('Replacing functions...')
dedup_train[body] = dedup_train[body].apply(lambda x:function_regex.sub(" function ",x))
test[body] = test[body].apply(lambda x:function_regex.sub(" function ",x))

print('Replacing issue numbers...')
dedup_train[title] = dedup_train[title].apply(lambda x:issue_regex.sub(" issue ",x))
dedup_train[body] = dedup_train[body].apply(lambda x:issue_regex.sub(" issue ",x))
test[title] = test[title].apply(lambda x:issue_regex.sub(" issue ",x))
test[body] = test[body].apply(lambda x:issue_regex.sub(" issue ",x))

print('Converting to lower case...')
dedup_train[title] = dedup_train[title].str.lower()
dedup_train[body] = dedup_train[body].str.lower()
test[title] = test[title].str.lower()
test[body] = test[body].str.lower()

Replacing functions...
Replacing issue numbers...
Converting to lower case...


remove extra information from text

In [13]:
print('Removing punctuations...')
replace_string = ' '*len(punctuations)
dedup_train[title] = dedup_train[title].str.translate(str.maketrans(punctuations, replace_string))
dedup_train[body] = dedup_train[body].str.translate(str.maketrans(punctuations, replace_string))
test[title] = test[title].str.translate(str.maketrans(punctuations, replace_string))
test[body] = test[body].str.translate(str.maketrans(punctuations, replace_string))

print('Removing non-ascii charachters...')
dedup_train[title] = dedup_train[title].apply(lambda x:re.sub(ascii_regex, '', x))
dedup_train[title] = dedup_train[title].apply(lambda x:ud.normalize('NFD', x))
dedup_train[body] = dedup_train[body].apply(lambda x:re.sub(ascii_regex, '', x))
dedup_train[body] = dedup_train[body].apply(lambda x:ud.normalize('NFD', x))

test[title] = test[title].apply(lambda x:re.sub(ascii_regex, '', x))
test[title] = test[title].apply(lambda x:ud.normalize('NFD', x))
test[body] = test[body].apply(lambda x:re.sub(ascii_regex, '', x))
test[body] = test[body].apply(lambda x:ud.normalize('NFD', x))

#print('Replacing fixed part of repo URl column...')
#dedup_train[repo] = dedup_train[repo].apply(lambda x: x.replace('https://api.github.com/repos/', ''))
#test[repo] = test[repo].apply(lambda x: x.replace('https://api.github.com/repos/', ''))

print('Replacing white spaces...')
dedup_train[title] = dedup_train[title].apply(lambda x:" ".join(x.split()))
dedup_train[body] = dedup_train[body].apply(lambda x:" ".join(x.split()))
test[title] = test[title].apply(lambda x:" ".join(x.split()))
test[body] = test[body].apply(lambda x:" ".join(x.split()))

Removing punctuations...
Removing non-ascii charachters...
Replacing white spaces...


check the number of tokens in the issue title and issue body

train

In [14]:
dedup_train['title_token_count'] = dedup_train[title].apply(lambda x: len(x.split()))
print(f'Average number of tokens in issue title: {dedup_train['title_token_count'].mean()}, Median of tokens in issue title: {dedup_train['title_token_count'].median()}')

Average number of tokens in issue title: 8.50552635770027, Median of tokens in issue title: 8.0


In [15]:
dedup_train['body_token_count'] = dedup_train[body].apply(lambda x: len(x.split()))
print(f'Average number of tokens in issue body: {dedup_train['body_token_count'].mean()}, Median of tokens in issue body: {dedup_train['body_token_count'].median()}')

Average number of tokens in issue body: 201.95992992514732, Median of tokens in issue body: 121.0


test

In [16]:
test['title_token_count'] = test[title].apply(lambda x: len(x.split()))
print(f'Average number of tokens in issue title: {test['title_token_count'].mean()}, Median of tokens in issue title: {test['title_token_count'].median()}')

Average number of tokens in issue title: 8.46225874259507, Median of tokens in issue title: 8.0


In [17]:
test['body_token_count'] = test[body].apply(lambda x: len(x.split()))
print(f'Average number of tokens in issue body: {test['body_token_count'].mean()}, Median of tokens in issue body: {test['body_token_count'].median()}')

Average number of tokens in issue body: 200.2710682209058, Median of tokens in issue body: 119.0


truncate columns

In [18]:
dedup_train[title] = dedup_train[title].apply(lambda x: ' '.join(x.split(maxsplit=max_title)[:max_title]))
dedup_train[body] = dedup_train[body].apply(lambda x: ' '.join(x.split(maxsplit=max_body)[:max_body]))
test[title] = test[title].apply(lambda x: ' '.join(x.split(maxsplit=max_title)[:max_title]))
test[body] = test[body].apply(lambda x: ' '.join(x.split(maxsplit=max_body)[:max_body]))

prepare labels column

In [19]:
dedup_train[label] = pd.Categorical(dedup_train[label])
test[label] = pd.Categorical(test[label])
dedup_train[label_col] = dedup_train[label].cat.codes
test[label_col] = test[label].cat.codes

concat issue columns in one "text" column to feed the BERT model

In [20]:
dedup_train[text_col] = 'time ' + dedup_train[time] + ' author ' + dedup_train[author] +' repo ' + dedup_train[repo] + ' title ' + dedup_train[title] + ' body ' + dedup_train[body]
test[text_col] = 'time ' + test[time] + ' author ' + test[author] +' repo ' + test[repo] + ' title ' + test[title] + ' body ' + test[body]

In [21]:
dedup_train.head(1)

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,preprocessed_title,user,labels,state,locked,assignee,assignees,milestone,comments,created_at,updated_at,closed_at,author_association,active_lock_reason,preprocessed_body,reactions,timeline_url,performed_via_github_app,state_reason,score,repository_name,issue/pr,created_at_timestamp,closed_at_timestamp,issue_resolution_time,open_since,labels_id,labels_node_id,labels_name,labels_color,labels_default,labels_description,mapLabel,original_issue_title,original_issue_body,title_token_count,body_token_count,labels_cat,text
71638,https://api.github.com/repos/Anuken/Mindustry/issues/2131,https://api.github.com/repos/Anuken/Mindustry,['https://api.github.com/repos/Anuken/Mindustry/labels/bug'],https://api.github.com/repos/Anuken/Mindustry/issues/2131/comments,https://api.github.com/repos/Anuken/Mindustry/issues/2131/events,https://github.com/Anuken/Mindustry/issues/2131,628551684,MDU6SXNzdWU2Mjg1NTE2ODQ=,2131,issue with drillspeed for laserdrills,"{'login': 'Absolute359', 'id': 66265700, 'node_id': 'MDQ6VXNlcjY2MjY1NzAw', 'avatar_url': 'https://avatars.githubusercontent.com/u/66265700?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/Absolute359', 'html_url': 'https://github.com/Absolute359', 'followers_url': 'https://api.github.com/users/Absolute359/followers', 'following_url': 'https://api.github.com/users/Absolute359/following{/other_user}', 'gists_url': 'https://api.github.com/users/Absolute359/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/Absolute359/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/Absolute359/subscriptions', 'organizations_url': 'https://api.github.com/users/Absolute359/orgs', 'repos_url': 'https://api.github.com/users/Absolute359/repos', 'events_url': 'https://api.github.com/users/Absolute359/events{/privacy}', 'received_events_url': 'https://api.github.com/users/Absolute359/received_events', 'type': 'User', 'site_admin': False}","[{'id': 594803818, 'node_id': 'MDU6TGFiZWw1OTQ4MDM4MTg=', 'url': 'https://api.github.com/repos/Anuken/Mindustry/labels/bug', 'name': 'bug', 'color': 'd30051', 'default': True, 'description': ''}]",closed,False,None,[],None,2,2020-06-01T16:01:49Z,2020-06-01T16:54:52Z,2020-06-01T16:54:52Z,NONE,None,platform windows 7 build steam build 104.10 issue a water boosted laserdrill on sand displays 4.94 s but it can't supply 8 oil extractors wich only need 4.8 sand s in total. there is no overflowing sand and the oil lake on the last extractor is smaller compared to the others. for smooth designs it would be nice to get the actuall 4.94 s out of that drill image https user images.githubusercontent.com 66265700 83427216 ac0add00 a430 11ea 9ea5 026092d03b3e.png place an x no spaces between the brackets to confirm that you have read the line below. x i have searched the closed and open issues to make sure that this problem has not already been reported.,"{'url': 'https://api.github.com/repos/Anuken/Mindustry/issues/2131/reactions', 'total_count': 0, '+1': 0, '-1': 0, 'laugh': 0, 'hooray': 0, 'confused': 0, 'heart': 0, 'rocket': 0, 'eyes': 0}",https://api.github.com/repos/Anuken/Mindustry/issues/2131/timeline,None,completed,1.0,Anuken/Mindustry,issue,2020-06-01 16:01:49,2020-06-01 16:54:52,0.03684,NaN,[594803818],['MDU6TGFiZWw1OTQ4MDM4MTg='],['bug'],['d30051'],[True],[''],bug,Issue with drillspeed for Laserdrills,**Platform**: *Windows 7*\r\n\r\n**Build**: *steam build 104.10*\r\n\r\n**Issue**: *A water boosted laserdrill on sand displays 4.94/s but it can't supply 8 oil extractors wich only need 4.8 sand/s in total. There is no overflowing sand and the oil lake on the last extractor is smaller compared to the others. For smooth designs it would be nice to get the actuall 4.94/s out of that drill*\r\n![image](https://user-images.githubusercontent.com/66265700/83427216-ac0add00-a430-11ea-9ea5-026092d03b3e.png)\r\n\r\n---\r\n\r\n*Place an X (no spaces) between the bra

split and save datasets

In [23]:
dedup_train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

dedup_train[['original_issue_title','preprocessed_title','original_issue_body','preprocessed_body','text','mapLabel','labels_cat']].to_csv(f'msc_dataset_preprocessing/train_clean_concat_{max_title+max_body}_detailed_data.csv', index = False)
test[['original_issue_title','preprocessed_title','original_issue_body','preprocessed_body','text','mapLabel','labels_cat']].to_csv(f'msc_dataset_preprocessing/test_clean_concat_{max_title+max_body}_detailed_data.csv', index = False)

In [24]:
dedup_train[['text','labels']].to_csv(f'msc_dataset_preprocessing/train_clean_concat_{max_title+max_body}.csv', index = False)
test[['text','labels']].to_csv(f'msc_dataset_preprocessing/test_clean_concat_{max_title+max_body}.csv', index = False)